In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Splitting 80% for training and 20% for validation
)

image_dir = '/content/drive/My Drive/DataSet_Work/Anaconda1/Train/ham'
Val_path = '/content/drive/My Drive/DataSet_Work/Anaconda1/Valid'

train_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Since it's a binary classification task
    subset='training'  # Specify this is the training set
)

validation_generator = train_datagen.flow_from_directory(
    Val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify this is the validation set
)

Found 0 images belonging to 0 classes.
Found 211 images belonging to 2 classes.


In [ ]:
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import os

# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

# Function to load an image and preprocess it
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array


# List to store extracted features
features_list = []

# Extract features for each image in the directory
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    img_array = load_and_preprocess_image(img_path)
    features = model.predict(img_array)
    features_list.append(features)

# Convert the list of features to a numpy array
features_array = np.array(features_list)
features_array = features_array.reshape(len(features_list), -1)  # Flatten the features if necessary

# Save the extracted features to a file
np.save('extracted_features.npy', features_array)

print("Features extracted and saved successfully.")

1/1 [==============================] - 0s 50ms/step
Features extracted and saved successfully.
